In [26]:
from Bio import SeqIO
from Bio import motifs
import numpy as np
import pandas as pd

In [27]:
# Read sequences from FASTQ file
fastq_file = "../barcode06_trimmed_direct.fastq"
sequences = [str(record.seq) for record in SeqIO.parse(fastq_file, "fastq")]

In [28]:
min_length = min(len(seq) for seq in sequences)
trimmed_sequences = [seq[:min_length] for seq in sequences]

In [29]:
m = motifs.create(trimmed_sequences)
pwm = m.counts.normalize()

In [30]:
print(pwm)

        0      1      2      3      4      5      6      7      8      9     10     11     12     13     14     15     16     17     18     19     20     21     22     23     24     25     26     27     28     29     30     31     32     33     34     35     36     37     38     39
A:   0.00   0.00   0.00   0.07   0.28   0.28   0.33   0.50   0.35   0.25   0.28   0.42   0.33   0.35   0.25   0.25   0.45   0.33   0.12   0.47   0.28   0.00   0.12   0.07   0.12   0.33   0.20   0.15   0.07   0.42   0.33   0.15   0.28   0.42   0.38   0.15   0.55   0.55   0.45   0.50
C:   0.15   0.23   0.35   0.57   0.42   0.15   0.12   0.33   0.30   0.12   0.17   0.12   0.15   0.25   0.40   0.47   0.12   0.28   0.33   0.20   0.35   0.40   0.33   0.33   0.40   0.28   0.20   0.12   0.28   0.17   0.12   0.40   0.40   0.25   0.07   0.23   0.17   0.12   0.05   0.05
G:   0.85   0.65   0.42   0.23   0.12   0.42   0.45   0.17   0.20   0.17   0.25   0.15   0.25   0.28   0.35   0.15   0.25   0.07   0.35   0.17   0.05  

In [31]:
def find_consensus_motifs(sequences, threshold=0.1):
    min_length = min(len(seq) for seq in sequences)
    trimmed_sequences = [seq[:min_length] for seq in sequences]

    def recursive_find_consensus_motifs(sequences, current_motif, position):
        if position == min_length:
            return [current_motif]

        nucleotide_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        for seq in sequences:
            nucleotide_counts[seq[position]] += 1

        max_count = max(nucleotide_counts.values())
        threshold_count = max_count * (1 - threshold)

        consensus_nucleotides = [nucleotide for nucleotide, count in nucleotide_counts.items() if count >= threshold_count]

        consensus_motifs = []
        for nucleotide in consensus_nucleotides:
            new_sequences = [seq for seq in sequences if seq[position] == nucleotide]
            new_motif = current_motif + nucleotide
            consensus_motifs.extend(recursive_find_consensus_motifs(new_sequences, new_motif, position + 1))

        return consensus_motifs

    return recursive_find_consensus_motifs(trimmed_sequences, "", 0)

In [32]:
def print_pwm(pwm):
    print("Position Weight Matrix:")
    print("        ", end="")
    for i in range(pwm.shape[1]):
        print(f"{i:6d}", end="")
    print()
    print("A: ", end="")
    for i in range(pwm.shape[1]):
        print(f"{pwm[0, i]:.2f}", end=" ")
    print()
    print("C: ", end="")
    for i in range(pwm.shape[1]):
        print(f"{pwm[1, i]:.2f}", end=" ")
    print()
    print("G: ", end="")
    for i in range(pwm.shape[1]):
        print(f"{pwm[2, i]:.2f}", end=" ")
    print()
    print("T: ", end="")
    for i in range(pwm.shape[1]):
        print(f"{pwm[3, i]:.2f}", end=" ")
    print()

In [33]:
consensus_motifs = find_consensus_motifs(sequences)


In [34]:
print("Consensus Motifs:")
for i, motif in enumerate(consensus_motifs):
    print(f"{i+1}: {motif}")

Consensus Motifs:
1: GGGCCGAAAGCTAGCACGACCTCTGTTCCAGGATCCATGA
2: GGGCCTGAGCTTACGAGTCTTCATGGATCCATGAAGACGA


In [35]:
pwm

{'A': (0.0,
  0.0,
  0.0,
  0.075,
  0.275,
  0.275,
  0.325,
  0.5,
  0.35,
  0.25,
  0.275,
  0.425,
  0.325,
  0.35,
  0.25,
  0.25,
  0.45,
  0.325,
  0.125,
  0.475,
  0.275,
  0.0,
  0.125,
  0.075,
  0.125,
  0.325,
  0.2,
  0.15,
  0.075,
  0.425,
  0.325,
  0.15,
  0.275,
  0.425,
  0.375,
  0.15,
  0.55,
  0.55,
  0.45,
  0.5),
 'C': (0.15,
  0.225,
  0.35,
  0.575,
  0.425,
  0.15,
  0.125,
  0.325,
  0.3,
  0.125,
  0.175,
  0.125,
  0.15,
  0.25,
  0.4,
  0.475,
  0.125,
  0.275,
  0.325,
  0.2,
  0.35,
  0.4,
  0.325,
  0.325,
  0.4,
  0.275,
  0.2,
  0.125,
  0.275,
  0.175,
  0.125,
  0.4,
  0.4,
  0.25,
  0.075,
  0.225,
  0.175,
  0.125,
  0.05,
  0.05),
 'G': (0.85,
  0.65,
  0.425,
  0.225,
  0.125,
  0.425,
  0.45,
  0.175,
  0.2,
  0.175,
  0.25,
  0.15,
  0.25,
  0.275,
  0.35,
  0.15,
  0.25,
  0.075,
  0.35,
  0.175,
  0.05,
  0.325,
  0.075,
  0.05,
  0.2,
  0.075,
  0.125,
  0.35,
  0.45,
  0.275,
  0.15,
  0.15,
  0.2,
  0.125,
  0.2,
  0.4,
  0.2,
  0.2,
  

In [36]:
# Create a motif
m = motifs.create(trimmed_sequences)

In [37]:
m.consensus

Seq('GGGCCGGAATTAAACCAAGACCTTCATTGATCCAAGAAAA')

In [38]:
m.pwm

{'A': (0.0,
  0.0,
  0.0,
  0.075,
  0.275,
  0.275,
  0.325,
  0.5,
  0.35,
  0.25,
  0.275,
  0.425,
  0.325,
  0.35,
  0.25,
  0.25,
  0.45,
  0.325,
  0.125,
  0.475,
  0.275,
  0.0,
  0.125,
  0.075,
  0.125,
  0.325,
  0.2,
  0.15,
  0.075,
  0.425,
  0.325,
  0.15,
  0.275,
  0.425,
  0.375,
  0.15,
  0.55,
  0.55,
  0.45,
  0.5),
 'C': (0.15,
  0.225,
  0.35,
  0.575,
  0.425,
  0.15,
  0.125,
  0.325,
  0.3,
  0.125,
  0.175,
  0.125,
  0.15,
  0.25,
  0.4,
  0.475,
  0.125,
  0.275,
  0.325,
  0.2,
  0.35,
  0.4,
  0.325,
  0.325,
  0.4,
  0.275,
  0.2,
  0.125,
  0.275,
  0.175,
  0.125,
  0.4,
  0.4,
  0.25,
  0.075,
  0.225,
  0.175,
  0.125,
  0.05,
  0.05),
 'G': (0.85,
  0.65,
  0.425,
  0.225,
  0.125,
  0.425,
  0.45,
  0.175,
  0.2,
  0.175,
  0.25,
  0.15,
  0.25,
  0.275,
  0.35,
  0.15,
  0.25,
  0.075,
  0.35,
  0.175,
  0.05,
  0.325,
  0.075,
  0.05,
  0.2,
  0.075,
  0.125,
  0.35,
  0.45,
  0.275,
  0.15,
  0.15,
  0.2,
  0.125,
  0.2,
  0.4,
  0.2,
  0.2,
  

In [39]:
# Create a position weight matrix (PWM)
pwm = m.counts.normalize()

In [40]:
# Find all possible consensus motifs
consensus_motifs = []
for i in range(min_length):
    nucleotide_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
    for seq in trimmed_sequences:
        nucleotide_counts[seq[i]] += 1
    max_count = max(nucleotide_counts.values())
    threshold = max_count * 0.5  # adjust this threshold as needed
    consensus_nucleotides = [nucleotide for nucleotide, count in nucleotide_counts.items() if count > threshold]
    consensus_motifs.append(consensus_nucleotides)

In [41]:
consensus_motifs

[['G'],
 ['G'],
 ['C', 'G', 'T'],
 ['C'],
 ['A', 'C'],
 ['A', 'G'],
 ['A', 'G'],
 ['A', 'C'],
 ['A', 'C', 'G'],
 ['A', 'T'],
 ['A', 'C', 'G', 'T'],
 ['A', 'T'],
 ['A', 'G', 'T'],
 ['A', 'C', 'G'],
 ['A', 'C', 'G'],
 ['A', 'C'],
 ['A', 'G'],
 ['A', 'C', 'T'],
 ['C', 'G', 'T'],
 ['A'],
 ['A', 'C', 'T'],
 ['C', 'G', 'T'],
 ['C', 'T'],
 ['C', 'T'],
 ['C', 'T'],
 ['A', 'C', 'T'],
 ['T'],
 ['G', 'T'],
 ['C', 'G'],
 ['A', 'G'],
 ['A', 'T'],
 ['C', 'T'],
 ['A', 'C'],
 ['A', 'C'],
 ['A', 'G', 'T'],
 ['C', 'G', 'T'],
 ['A'],
 ['A'],
 ['A', 'G'],
 ['A', 'G']]

In [42]:
# Create a PWM for each consensus motif
pwms = []
for i in range(min_length):
    pwm = np.zeros((4, 1))
    for seq in trimmed_sequences:
        if seq[i] == 'A':
            pwm[0, 0] += 1
        elif seq[i] == 'C':
            pwm[1, 0] += 1
        elif seq[i] == 'G':
            pwm[2, 0] += 1
        elif seq[i] == 'T':
            pwm[3, 0] += 1
    pwm = pwm / len(trimmed_sequences)
    pwms.append(pwm)

In [43]:
pwms

[array([[0.  ],
        [0.15],
        [0.85],
        [0.  ]]),
 array([[0.   ],
        [0.225],
        [0.65 ],
        [0.125]]),
 array([[0.   ],
        [0.35 ],
        [0.425],
        [0.225]]),
 array([[0.075],
        [0.575],
        [0.225],
        [0.125]]),
 array([[0.275],
        [0.425],
        [0.125],
        [0.175]]),
 array([[0.275],
        [0.15 ],
        [0.425],
        [0.15 ]]),
 array([[0.325],
        [0.125],
        [0.45 ],
        [0.1  ]]),
 array([[0.5  ],
        [0.325],
        [0.175],
        [0.   ]]),
 array([[0.35],
        [0.3 ],
        [0.2 ],
        [0.15]]),
 array([[0.25 ],
        [0.125],
        [0.175],
        [0.45 ]]),
 array([[0.275],
        [0.175],
        [0.25 ],
        [0.3  ]]),
 array([[0.425],
        [0.125],
        [0.15 ],
        [0.3  ]]),
 array([[0.325],
        [0.15 ],
        [0.25 ],
        [0.275]]),
 array([[0.35 ],
        [0.25 ],
        [0.275],
        [0.125]]),
 array([[0.25],
        [0.4

In [44]:
# Build a tree structure to display as a Sankey diagram
nodes = []
edges = []
for i in range(min_length):
    for nucleotide in consensus_motifs[i]:
        node_id = f"{i}_{nucleotide}"
        nodes.append({"id": node_id, "label": nucleotide})
        if i > 0:
            for prev_nucleotide in consensus_motifs[i-1]:
                prev_node_id = f"{i-1}_{prev_nucleotide}"
                edges.append({"source": prev_node_id, "target": node_id, "value": 1})

In [45]:
# Convert nodes and edges to pandas DataFrames
nodes_df = pd.DataFrame(nodes)
edges_df = pd.DataFrame(edges)

In [46]:
# Save nodes and edges to CSV files
nodes_df.to_csv("nodes.csv", index=False)
edges_df.to_csv("edges.csv", index=False)

In [47]:
# Print consensus motifs and PWMs
for i in range(min_length):
    print(f"Position {i+1}:")
    print("Consensus Motifs:", consensus_motifs[i])
    print("Position Weight Matrix:")
    print(pwms[i])
    print()

Position 1:
Consensus Motifs: ['G']
Position Weight Matrix:
[[0.  ]
 [0.15]
 [0.85]
 [0.  ]]

Position 2:
Consensus Motifs: ['G']
Position Weight Matrix:
[[0.   ]
 [0.225]
 [0.65 ]
 [0.125]]

Position 3:
Consensus Motifs: ['C', 'G', 'T']
Position Weight Matrix:
[[0.   ]
 [0.35 ]
 [0.425]
 [0.225]]

Position 4:
Consensus Motifs: ['C']
Position Weight Matrix:
[[0.075]
 [0.575]
 [0.225]
 [0.125]]

Position 5:
Consensus Motifs: ['A', 'C']
Position Weight Matrix:
[[0.275]
 [0.425]
 [0.125]
 [0.175]]

Position 6:
Consensus Motifs: ['A', 'G']
Position Weight Matrix:
[[0.275]
 [0.15 ]
 [0.425]
 [0.15 ]]

Position 7:
Consensus Motifs: ['A', 'G']
Position Weight Matrix:
[[0.325]
 [0.125]
 [0.45 ]
 [0.1  ]]

Position 8:
Consensus Motifs: ['A', 'C']
Position Weight Matrix:
[[0.5  ]
 [0.325]
 [0.175]
 [0.   ]]

Position 9:
Consensus Motifs: ['A', 'C', 'G']
Position Weight Matrix:
[[0.35]
 [0.3 ]
 [0.2 ]
 [0.15]]

Position 10:
Consensus Motifs: ['A', 'T']
Position Weight Matrix:
[[0.25 ]
 [0.125]
 [